# Fitting ellipsoids to labelled volume data

This notebook demonstrates how to fit ellipsoids to labelled volume data and then
compare the ellipsoids using various calculations and statistics.

First get a labelled volume from the example data (or somewhere else):

In [1]:
from mouse_embryo_labeller import tools
folder = "../" + tools.EXAMPLE_FOLDER
folder

'../../example_data/'

In [2]:
nc = tools.get_example_nucleus_collection(folder)
tsc = tools.get_example_timestamp_collection(folder, nc)
tsc.id_to_timestamp.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [3]:
ts = tsc.id_to_timestamp[10]
ts.load_truncated_arrays()
label_array = ts.l3d_truncated
label_array.shape

(71, 180, 247)

In [4]:
# The label array contains a small number of labels in a numpy int array:
import numpy as np
np.unique(label_array)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [5]:
# View the raw array.  Slide the slider to move between layers.
from jp_doodle import array_image
array_image.show_arrays(label_array)

In [6]:
# Create an array fitter object for this array and adjust the coordinates
# so the array maps to a geometric unit cube, using di, dj, dk.

from mouse_embryo_labeller.ellipsoid_fit import ArrayFitter

(I, J, K) = label_array.shape
AF = ArrayFitter(label_array, di=1.0/I, dj=1.0/J, dk=1.0/K)

In [8]:
# Fit all labels (>0) inside a single ellipse and display a jupyter widgets graphic of the fit.
# Use 1000 randomly chosen test points for the fit.

info = AF.fit_ellipse_to_range(lower_limit=1, upper_limit=None, display=True, point_limit=1000)

In [9]:
# Drag the mouse in the generated graphic to rotate the figures.

In [12]:
print("Statistics")
print("axes", info.axes())
print("axis lengths", info.axis_lengths())
print("volume", info.volume(), "surface_area", info.surface_area(), "sphericity", info.sphericity())

Statistics
axes [[ 0.10917607 -0.05857698  0.10610783]
 [ 0.12329408  0.27619463  0.02561436]
 [-0.12865482  0.04295615  0.15608903]]
axis lengths [0.16312433469574755, 0.3035473553506786, 0.20678751833435285]
volume 0.042890206520850585 surface_area 0.622710547657092 sphericity 0.9515610585580799


In [13]:
# Fit ellipses to each labelled region separately (don't display widgets).

label_to_info = {}
for i in np.unique(label_array):
    if i > 0:
        label_to_info[i] = AF.fit_ellipse_to_range(lower_limit=i, upper_limit=i, point_limit=1000)

In [16]:
# Show relationship of each label ellipse to the ellipse containing all labels.
for i in sorted(label_to_info):
    label_info = label_to_info[i]
    print(i, "offset", label_info.relative_offset_to_center_of(info),
         "inside?", label_info.proportion_inside_of(info))

1 offset 0.7151036349725067 inside? 0.9713114754098361
2 offset 0.8548151649589097 inside? 0.9528688524590164
3 offset 0.7426537633182846 inside? 1
4 offset 0.8054360679127269 inside? 0.9631147540983607
5 offset 0.7941382557870603 inside? 0.9877049180327869
6 offset 0.8357816184771855 inside? 0.9467213114754098
7 offset 0.7152937414775216 inside? 0.9692622950819673
8 offset 0.8792028253682886 inside? 0.8545081967213115


In [15]:
print(info.relative_offset_to_center_of.__doc__)


        Linear measure of how near the center of this ellipse is to the center of the other
        ellipse.  Returns 0 at the center, 1 at the boundary of the ellipse and > 1 outside the ellipse.
        


In [17]:
print(label_info.proportion_inside_of.__doc__)


        Return approximate proportion of this ellipse inside the other ellipse by relative volume.
        Returns 1 if completely inside and 0 if there is (nearly) no intersection.
        
